In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = \
        os.environ['CUDA_VISIBLE_DEVICES'].split(',')[0]
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


0
Wed Oct 25 15:48:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   32C    P0    71W / 300W |  14426MiB / 32510MiB |     81%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A 

In [2]:
import os
import json
import argparse
import logging
import random
import numpy as np
import torch
import datasets
from alpaca_eval import evaluate as alpaca_farm_evaluate
from eval.utils import query_openai_chat_model, query_openai_model, generate_completions, dynamic_import_function, load_hf_lm_and_tokenizer

[2023-10-25 15:48:52,450] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
# import openai
# openai.api_key = os.environ['OPENAI_API_KEY']

# response = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo-0301",
#   messages=[{'role': 'user', 'content': 'Who is the president of the US right now.'}],
#   temperature=0,
#   max_tokens=256
# )

In [118]:
parser = argparse.ArgumentParser()
parser.add_argument("--reference_path", type=str, default="data/eval/alpaca_farm/davinci_003_outputs_2048_token.json", help="Path to the reference outputs. Alpaca_eval leaderboard use davinci_003 to generate the reference outputs, but they limit the max_tokens to 300. Here regenerated reference outputs with max_tokens=2048.")
parser.add_argument("--save_dir", type=str, default="results/alpaca_farm")
parser.add_argument("--model_name_or_path", type=str, default=None, help="If specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="If specified, we will load the tokenizer from here.")
parser.add_argument("--openai_engine", type=str, default=None, help="If specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="Batch size for evaluation.")
parser.add_argument("--load_in_8bit", action="store_true", help="Load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, we will use the chat format for the prompts.")
parser.add_argument("--chat_formatting_function", type=str, default="eval.templates.create_prompt_with_tulu_chat_format", help="The function to use to create the chat format. This function will be dynamically imported. Please see examples in `eval/templates.py`.")
parser.add_argument("--use_vllm", action="store_true", help="If given, we will use vLLM to generate the predictions - much faster.")
parser.add_argument("--annotators_config", type=str, default="alpaca_eval_gpt4_0314")
parser.add_argument("--max_num_examples", type=int, default=None, help="maximum number of examples to evaluate.")


model_name_or_path = '../results/baselines/huggyllama/llama-7b'
model_name_or_path = '../results/ft1_ep=1/llama-7b_tuluv1m'
save_dir = os.path.join(model_name_or_path, 'eval/alpaca_farm/')

# eval_batch_size <=5 without oom for llama-7b gen 2048 new tokens

#     --use_chat_format \
#     --max_num_examples 50 \
cmd = f"""
    --reference_path alpaca_eval_data \
    --model_name_or_path {model_name_or_path} \
    --save_dir {save_dir} \
    --eval_batch_size 5 \
    --use_chat_format \
    --annotators_config chatgpt \
    --max_num_examples 5 \
"""

args = jpt_parse_args(parser, cmd)
args

Namespace(reference_path='alpaca_eval_data', save_dir='../results/ft1_ep=1/llama-7b_tuluv1m/eval/alpaca_farm/', model_name_or_path='../results/ft1_ep=1/llama-7b_tuluv1m', tokenizer_name_or_path=None, openai_engine=None, eval_batch_size=5, load_in_8bit=False, gptq=False, use_chat_format=True, chat_formatting_function='eval.templates.create_prompt_with_tulu_chat_format', use_vllm=False, annotators_config='chatgpt', max_num_examples=5)

In [127]:

random.seed(42)
os.makedirs(args.save_dir, exist_ok=True)

if args.reference_path != 'alpaca_eval_data':
    raise NotImplementedError('Only support alpaca_eval_data for now. Need to work with `max_num_examples` in case I want to use other reference generation.')

logging.info("loading data and model...")
alpaca_eval_data = datasets.load_dataset("tatsu-lab/alpaca_farm", "alpaca_farm_evaluation")["eval"]

if args.max_num_examples and len(alpaca_eval_data) > args.max_num_examples:
    inds = random.sample(range(len(alpaca_eval_data)), args.max_num_examples)
    alpaca_eval_data = alpaca_eval_data.select(inds)

prompts = []
chat_formatting_function = dynamic_import_function(args.chat_formatting_function) if args.use_chat_format else None
for example in alpaca_eval_data:
    prompt = example["instruction"] + "\n\n" + example["input"] if example["input"] != "" else example["instruction"]
    if args.use_chat_format:
        messages = [{"role": "user", "content": prompt}]
        prompt = chat_formatting_function(messages, add_bos=False)
    prompts.append(prompt)


INFO:root:loading data and model...


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
model, tokenizer = load_hf_lm_and_tokenizer(
    model_name_or_path=args.model_name_or_path,
    tokenizer_name_or_path=args.tokenizer_name_or_path if args.tokenizer_name_or_path is not None else args.model_name_or_path,
    load_in_8bit=args.load_in_8bit,
    device_map="balanced_low_0" if torch.cuda.device_count() > 1 else "auto",
    gptq_model=args.gptq,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
outputs = generate_completions(
    model=model,
    tokenizer=tokenizer,
    prompts=prompts[:5],
    max_new_tokens=2048,
    do_sample=False,
    temperature=0,
    batch_size=args.eval_batch_size if args.eval_batch_size else 1,
)


Generating Completions: 100%|██████████| 5/5 [02:07<00:00, 25.48s/it]


In [43]:
for i in range(len(outputs)):
    print(i, )
    prompt_len = len(tokenizer(prompts[i], add_special_tokens=False)['input_ids'])
    print(f'---------- user (length={prompt_len}) ----------')
    print(prompts[i])
    output_len = len(tokenizer(outputs[i], add_special_tokens=True)['input_ids'])
    print(f'---------- assistant (length={output_len}) ----------')
    print(outputs[i])
    print('\n\n\n')
    

In [44]:

model_name = os.path.basename(os.path.normpath(args.model_name_or_path)) if args.model_name_or_path is not None else args.openai_engine
model_results = []
with open(os.path.join(args.save_dir, f"{model_name}-greedy-long-output.json"), "w") as fout:
    for example, output in zip(alpaca_eval_data, outputs):
        example["output"] = output
        example["generator"] = f"{model_name}-greedy-long"
        example["sample_mode"] = "temp=0,max_new_tokens=2048"
        fout.write(json.dumps(example) + "\n")
        model_results.append(example)

In [21]:
df_leaderboard, annotations = alpaca_farm_evaluate(
    model_outputs=model_results,
    reference_outputs=args.reference_path if args.reference_path != 'alpaca_eval_data' else alpaca_eval_data,
    annotators_config=args.annotators_config,
    output_path=args.save_dir,
    is_return_instead_of_print=True,
)

INFO:root:Evaluating the llama-7b_tuluv1m-greedy-long outputs.
INFO:root:Creating the annotator from `chatgpt`.
INFO:root:Saving annotations to `/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt/annotations_seed0_configs.json`.
Annotation chunk:   0%|          | 0/1 [00:00<?, ?it/s]INFO:root:Annotating 5 examples with chatgpt
INFO:root:Using `openai_completions` on 5 prompts using gpt-3.5-turbo-0301.
INFO:root:Kwargs to completion: {'n': 1, 'model': 'gpt-3.5-turbo-0301', 'is_chat': True, 'temperature': 0}. num_procs=5


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 


prompt_batches: 100%|██████████| 5/5 [00:00<00:00,  7.46it/s]
INFO:root:Completed 5 examples in 0.9 seconds.
INFO:root:Saving all annotations to /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt/annotations_seed0_configs.json.
Annotation chunk: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]
INFO:root:Saving all results to ../results/ft1_ep=1/llama-7b_tuluv1m/eval/alpaca_farm
INFO:root:Not saving the result to the cached leaderboard because precomputed_leaderboard is not a path but <class 'NoneType'>.


In [158]:

# prices = [x['price_per_example'] for x in annotations]
# print(f'Price (per-example / total) = {np.mean(prices):.4f} / {np.sum(prices):.2f}')

# times = [x['time_per_example'] for x in annotations]
# print(f'Time  (per-example / total) = {np.mean(times):.4f} / {np.sum(times):.2f}')

# df_leaderboard['avg_output_tok_length'] = np.mean(
#     [len(tokenizer(x['output'])['input_ids']) for x in model_results])
# df_leaderboard['price'] = np.sum(prices)

# df_leaderboard.insert(0, 'model', df_leaderboard.index)
df_leaderboard.reset_index(drop=True, inplace=True)

# print(df_leaderboard.to_string(float_format="%.2f"))
display(df_leaderboard)
# save to json

with open(os.path.join(args.save_dir, f"metrics.json"), "w") as fout:
    json.dump(df_leaderboard.iloc[0].to_dict(), fout)

,model,win_rate,standard_error,n_wins,n_wins_base,n_draws,n_total,mode,avg_length,avg_output_tok_length,price
0,llama-7b_tuluv1m-greedy-long,60.0,24.494897,3.0,2.0,0.0,5.0,community,2238.0,694.4,0.008982


In [154]:
df_leaderboard

,model,win_rate,standard_error,n_wins,n_wins_base,n_draws,n_total,mode,avg_length,avg_output_tok_length,price
llama-7b_tuluv1m-greedy-long,llama-7b_tuluv1m-greedy-long,60.0,24.494897,3.0,2.0,0.0,5.0,community,2238.0,694.4,0.008982
